# Multi-disentangled-features Gaussian Processes Variational AutoEncoder (MGP-VAE)

In [2]:
import torch
import torch.nn as nn
from torchvision.utils import save_image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import argparse
import os
from itertools import cycle
from torch.utils.data import DataLoader
#from tensorboard import SummaryWriter
from torchvision.utils import save_image

from MGP_VAE.vid_process import resize_cropped, resize_keepAR, resize_mnist
from MGP_VAE.flags import *
from MGP_VAE.networks import Encoder, Decoder
from MGP_VAE.utils import weights_init, mse_loss, plot_image, plot_training_images
from MGP_VAE.covariance_fns import *
from MGP_VAE.flags import *
from MGP_VAE.setup_priors import *
from MGP_VAE.dataloader import *

from dataset_utils import GenericDataset, SpatiotemporalDataset

torch.cuda.empty_cache()

usage: ipykernel_launcher.py [-h] [--cuda CUDA] [--dataset DATASET]
                             [--batch_size BATCH_SIZE]
                             [--test_batch_size TEST_BATCH_SIZE]
                             [--image_size IMAGE_SIZE]
                             [--num_channels NUM_CHANNELS]
                             [--num_frames NUM_FRAMES] [--num_dim NUM_DIM]
                             [--num_fea NUM_FEA] [--fea FEA]
                             [--zero_mean_fea ZERO_MEAN_FEA]
                             [--mean_fea_s MEAN_FEA_S]
                             [--mean_fea_e MEAN_FEA_E] [--keep_rho KEEP_RHO]
                             [--beta BETA] [--lrate LRATE] [--beta_1 BETA_1]
                             [--beta_2 BETA_2] [--encoder_save ENCODER_SAVE]
                             [--decoder_save DECODER_SAVE]
                             [--log_file LOG_FILE] [--load_saved LOAD_SAVED]
                             [--start_epoch START_EPOCH]
                      

SystemExit: 2

In [ ]:
N_SAMPLES = 10000
N_CHANNELS = 1
IM_HEIGHT = 64
IM_WIDTH = 64
FORECAST_HORIZON = 10
LAGS = 10
MNIST = False

if MNIST:
    
    X = np.load("data/mnist_test_seq.npy").swapaxes(0,1)
    X = np.expand_dims(X, 2)
    trainDataset = GenericDataset(X[0:700], FORECAST_HORIZON, LAGS)
    validDataset = GenericDataset(X[700:], FORECAST_HORIZON, LAGS)
    
else:
    
    X = np.random.normal(0, 1, (N_SAMPLES, N_CHANNELS, IM_HEIGHT, IM_WIDTH)).astype(np.float32)
    trainDataset = SpatiotemporalDataset(X, FORECAST_HORIZON, LAGS)
    validDataset = SpatiotemporalDataset(X, FORECAST_HORIZON, LAGS)